In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [4]:
!ls /nfs/turbo/umms-awaljee/Cohort_ALL_deidentified_to_UMICH/Cohort_ALL_deidentified/DeepLearning/DataPreprocess/

Cohort	Longitudinal  merged_time_varying_data	Read me.txt


In [3]:
path = '/nfs/turbo/umms-awaljee/Cohort_ALL_deidentified_to_UMICH/Cohort_ALL_deidentified/DeepLearning/DataPreprocess/'

In [5]:
!more /nfs/turbo/umms-awaljee/Cohort_ALL_deidentified_to_UMICH/Cohort_ALL_deidentified/DeepLearning/DataPreprocess/Read\ me.txt

1, Original data in csv format
--> Folder "Longitudinal" + "Cohort"

2, Unique pair (PatientID, Time) with 20 labs and 11 treatment type (Ordered by 
Patient ID at first and then by Time).
--> Folder "merged_time_varying_data" --> File "MergedData.csv"

Note, the size of merged data is around 1GB. You could use function `fread()` in
 package `data.table` to load it into R using 20 seconds. 

Data Details: 10987831 rows and 33 columns, use blank "" for missing.
"PatientID", "Time", 20 labs, 11 types of treatment. For example, if treatment "
BOCEPREVIR" = 1, it means that the patient took this medication on this visit ti
me. It was calculated by the table "antiviral_rxfills" to check whether this vis
it time locates in the interval [DaysToDispend, DaysToDispend + DaysToSupply].

I remove 1772 patients with negative treatment time (118078 records). 



In [6]:
dat = pd.read_csv(path+'merged_time_varying_data/MergedData.csv')

In [19]:
dat.head()

,PatientID,Time,labalbumin,labalkalinephosphataseratio,labalphafetoproteinratio,labaltratio,labastratio,labbilirubin,labbloodureanitro,labchloride,...,DACLATASVIR,EPCLUSA,HARVONI,INTERFERON,PEGASYS,SIMEPREVIR,SOFOSBUVIR,TELAPREVIR,VIEKIRA,ZEPATIER
0,000010BE-20C2,0,4.6,0.968,NaN,NaN,1.0667,0.5,14.0,100.0,...,0,0,0,0,0,0,0,0,0,0
1,000010BE-20C2,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,000010BE-20C2,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,000010BE-20C2,312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,000010BE-20C2,361,4.7,0.912,NaN,1.4,0.9778,0.5,11.0,99.0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
dat.shape

(10987831, 33)

In [8]:
dat['PatientID'].value_counts().describe()

count    154628.000000
mean         71.059776
std          90.399444
min           2.000000
25%          26.000000
50%          50.000000
75%          87.000000
max        3912.000000
Name: PatientID, dtype: float64

In [22]:
dat.isna().apply(sum, axis=0)/len(dat)

PatientID                      0.000000
Time                           0.000000
labalbumin                     0.630693
labalkalinephosphataseratio    0.637866
labalphafetoproteinratio       0.948254
labaltratio                    0.640166
labastratio                    0.633857
labbilirubin                   0.628024
labbloodureanitro              0.505036
labchloride                    0.519027
labcreatinine                  0.499947
labglucose                     0.416291
labhemoglobin                  0.513398
labinr                         0.837040
labplt                         0.538871
labpotassium                   0.501823
labsodium                      0.504065
labtotprotein                  0.678069
labwbc                         0.542640
scoreapri                      0.574477
scoreastalt                    0.647892
scorebmi                       0.556354
BOCEPREVIR                     0.000000
DACLATASVIR                    0.000000
EPCLUSA                        0.000000


In [24]:
dat.columns

Index(['PatientID', 'Time', 'labalbumin', 'labalkalinephosphataseratio',
       'labalphafetoproteinratio', 'labaltratio', 'labastratio',
       'labbilirubin', 'labbloodureanitro', 'labchloride', 'labcreatinine',
       'labglucose', 'labhemoglobin', 'labinr', 'labplt', 'labpotassium',
       'labsodium', 'labtotprotein', 'labwbc', 'scoreapri', 'scoreastalt',
       'scorebmi', 'BOCEPREVIR', 'DACLATASVIR', 'EPCLUSA', 'HARVONI',
       'INTERFERON', 'PEGASYS', 'SIMEPREVIR', 'SOFOSBUVIR', 'TELAPREVIR',
       'VIEKIRA', 'ZEPATIER'],
      dtype='object')

In [25]:
labs = ['labalbumin', 'labalkalinephosphataseratio',
       'labalphafetoproteinratio', 'labaltratio', 'labastratio',
       'labbilirubin', 'labbloodureanitro', 'labchloride', 'labcreatinine',
       'labglucose', 'labhemoglobin', 'labinr', 'labplt', 'labpotassium',
       'labsodium', 'labtotprotein', 'labwbc', 'scoreapri', 'scoreastalt',
       'scorebmi']

In [27]:
treatment = ['BOCEPREVIR', 'DACLATASVIR', 'EPCLUSA', 'HARVONI',
       'INTERFERON', 'PEGASYS', 'SIMEPREVIR', 'SOFOSBUVIR', 'TELAPREVIR',
       'VIEKIRA', 'ZEPATIER']

In [28]:
(dat[treatment] == 1).apply(sum, axis=0)/len(dat)

BOCEPREVIR     0.003489
DACLATASVIR    0.000357
EPCLUSA        0.000644
HARVONI        0.010306
INTERFERON     0.001783
PEGASYS        0.031077
SIMEPREVIR     0.001159
SOFOSBUVIR     0.004040
TELAPREVIR     0.000349
VIEKIRA        0.002213
ZEPATIER       0.001435
dtype: float64